Thanks to Maarten's BERTopic  https://github.com/MaartenGr/BERTopic/

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
movie_plots_path = "/content/drive/My Drive/Practical Introduction to NLP Course/Module 3/American_Movie_Plots_2005_2021_v1_embeddings.csv"

In [3]:
import pandas as pd 
import ast
df = pd.read_csv(movie_plots_path,converters={'full_embeddings':ast.literal_eval}) 
print (df.shape)
df.head()

(3379, 5)


,Unnamed: 0,Movie Name,Wiki Link,Plot,full_embeddings
0,0,White Noise,https://en.wikipedia.org//wiki/White_Noise_(20...,Jonathan Rivers is an architect and lives with...,"[-0.0726638212800026, 0.05636344477534294, 0.0..."
1,1,Coach Carter,https://en.wikipedia.org//wiki/Coach_Carter,"Ken Carter lives in Richmond, California. He b...","[-0.0666189044713974, 0.04147614911198616, 0.0..."
2,2,Elektra,https://en.wikipedia.org//wiki/Elektra_(2005_f...,"After being killed in Daredevil, Elektra Natch...","[-0.03511177748441696, -0.01160984206944704, 0..."
3,3,Racing Stripes,https://en.wikipedia.org//wiki/Racing_Stripes,"During a thunderstorm, a traveling circus, Cir...","[-0.05486368015408516, 0.029759129509329796, -..."
4,4,Tom and Jerry: Blast Off to Mars,https://en.wikipedia.org//wiki/Tom_and_Jerry:_...,Tom (voiced by Bill Kopp) chases Jerry as usua...,"[0.010051962919533253, 0.06563024967908859, 0...."


In [4]:
!pip install --quiet sentence-transformers==2.2.0
!pip install --quiet textwrap3==0.9.2
!pip install --quiet umap-learn==0.5.3
!pip install --quiet hdbscan==0.8.28
!pip install pyyaml==5.4.1

     |████████████████████████████████| 79 kB 5.1 MB/s 
     |████████████████████████████████| 4.9 MB 15.0 MB/s 
     |████████████████████████████████| 1.3 MB 49.1 MB/s 
     |████████████████████████████████| 120 kB 45.4 MB/s 
     |████████████████████████████████| 6.6 MB 37.4 MB/s 
     |████████████████████████████████| 88 kB 4.8 MB/s 
     |████████████████████████████████| 1.1 MB 36.4 MB/s 
     |████████████████████████████████| 5.2 MB 7.6 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 636 kB 7.5 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 6.0
    Uninstalling PyYAML-6.0:
      Successfully uninstalled PyYAML-6.0


In [5]:
# movie_plots = df['Plot'].tolist()
# embeddings_movies = sentence_model.encode(movie_plots, show_progress_bar=True)
embeddings_movies = df['full_embeddings'].tolist()

In [10]:
import umap
import hdbscan
import numpy as np

# embeddings - np.array
umap_embeddings_5dim = umap.UMAP(n_neighbors=30, 
                            n_components=5, 
                            min_dist=0.0,metric='cosine').fit_transform(embeddings_movies)

umap_embeddings_2dim = umap.UMAP(n_neighbors=30, 
                            n_components=2, 
                            min_dist=0.0,metric='cosine').fit_transform(embeddings_movies)



In [11]:
cluster_movies = hdbscan.HDBSCAN(min_cluster_size=30,
                          metric='euclidean',                      
                          cluster_selection_method='eom').fit(umap_embeddings_5dim)

print (set(cluster_movies.labels_))
print (len(set(cluster_movies.labels_)))

{0, 1, 2, 3, 4, 5, 6, -1}
8


In [12]:
from collections import Counter
Counter(cluster_movies.labels_)

Counter({-1: 2069, 2: 601, 1: 144, 0: 87, 4: 83, 6: 212, 3: 109, 5: 74})

In [13]:
df["umap_embeddings_5dim"]  = umap_embeddings_5dim.tolist()
df["umap_embeddings_2dim"]  = umap_embeddings_2dim.tolist()
df["cluster labels"]  = cluster_movies.labels_
df.head()

,Unnamed: 0,Movie Name,Wiki Link,Plot,full_embeddings,umap_embeddings_5dim,umap_embeddings_2dim,cluster labels
0,0,White Noise,https://en.wikipedia.org//wiki/White_Noise_(20...,Jonathan Rivers is an architect and lives with...,"[-0.0726638212800026, 0.05636344477534294, 0.0...","[17.979957580566406, 0.06909535080194473, 14.6...","[17.152328491210938, -0.18462996184825897]",-1
1,1,Coach Carter,https://en.wikipedia.org//wiki/Coach_Carter,"Ken Carter lives in Richmond, California. He b...","[-0.0666189044713974, 0.04147614911198616, 0.0...","[19.398845672607422, 0.8634682297706604, 15.32...","[18.187257766723633, -0.751857340335846]",2
2,2,Elektra,https://en.wikipedia.org//wiki/Elektra_(2005_f...,"After being killed in Daredevil, Elektra Natch...","[-0.03511177748441696, -0.01160984206944704, 0...","[18.37731170654297, -0.19560715556144714, 13.2...","[13.422314643859863, 0.4871552884578705]",1
3,3,Racing Stripes,https://en.wikipedia.org//wiki/Racing_Stripes,"During a thunderstorm, a traveling circus, Cir...","[-0.05486368015408516, 0.029759129509329796, -...","[19.10247039794922, 0.8358369469642639, 13.432...","[13.882101058959961, -0.6579449772834778]",0
4,4,Tom and Jerry: Blast Off to Mars,https://en.wikipedia.org//wiki/Tom_and_Jerry:_...,Tom (voiced by Bill Kopp) chases Jerry as usua...,"[0.010051962919533253, 0.06563024967908859, 0....","[18.755529403686523, 0.9130203127861023, 14.69...","[15.446535110473633, -2.076817035675049]",-1


In [14]:
all_plots = df['Plot'].tolist()


In [ ]:
!python -m spacy download en_core_web_md

In [16]:
import spacy
nlp = spacy.load('en_core_web_md')
import tqdm.notebook as tq
all_plots_names_removed=[]
for p in tq.tqdm(all_plots):
      # Remove the tokens that you don't want.
      d = nlp(p)
      plot_annotated = [tok.text for tok in d if not
                                  (tok.is_punct or tok.is_stop 
                                    or tok.is_space or tok.pos_ == "PROPN" )
                                  ]
      # plot_annotated = [tok.text for tok in d if (tok.pos_ == "NOUN" or tok.pos_ == "VERB") ]
                                  
      plot_annotated_string = ' '.join(plot_annotated)                         
      all_plots_names_removed.append(plot_annotated_string)

  0%|          | 0/3379 [00:00<?, ?it/s]

In [17]:
df['filteredplot']=all_plots_names_removed
df.head()

,Unnamed: 0,Movie Name,Wiki Link,Plot,full_embeddings,umap_embeddings_5dim,umap_embeddings_2dim,cluster labels,filteredplot
0,0,White Noise,https://en.wikipedia.org//wiki/White_Noise_(20...,Jonathan Rivers is an architect and lives with...,"[-0.0726638212800026, 0.05636344477534294, 0.0...","[17.979957580566406, 0.06909535080194473, 14.6...","[17.152328491210938, -0.18462996184825897]",-1,architect lives wife unexpected death Eventual...
1,1,Coach Carter,https://en.wikipedia.org//wiki/Coach_Carter,"Ken Carter lives in Richmond, California. He b...","[-0.0666189044713974, 0.04147614911198616, 0.0...","[19.398845672607422, 0.8634682297706604, 15.32...","[18.187257766723633, -0.751857340335846]",2,lives coach basketball team having played team...
2,2,Elektra,https://en.wikipedia.org//wiki/Elektra_(2005_f...,"After being killed in Daredevil, Elektra Natch...","[-0.03511177748441696, -0.01160984206944704, 0...","[18.37731170654297, -0.19560715556144714, 13.2...","[13.422314643859863, 0.4871552884578705]",1,killed revived blind martial arts teaches anci...
3,3,Racing Stripes,https://en.wikipedia.org//wiki/Racing_Stripes,"During a thunderstorm, a traveling circus, Cir...","[-0.05486368015408516, 0.029759129509329796, -...","[19.10247039794922, 0.8358369469642639, 13.432...","[13.882101058959961, -0.6579449772834778]",0,thunderstorm traveling circus accidentally lea...
4,4,Tom and Jerry: Blast Off to Mars,https://en.wikipedia.org//wiki/Tom_and_Jerry:_...,Tom (voiced by Bill Kopp) chases Jerry as usua...,"[0.010051962919533253, 0.06563024967908859, 0....","[18.755529403686523, 0.9130203127861023, 14.69...","[15.446535110473633, -2.076817035675049]",-1,voiced chases usual house town arrive astronau...


In [18]:
docs_df_movies = pd.DataFrame(df, columns=["filteredplot"])
docs_df_movies['Topic'] = cluster_movies.labels_
docs_df_movies = docs_df_movies.groupby(['Topic'], as_index = False).agg({'filteredplot': ' '.join})

docs_df_movies.head(20)

,Topic,filteredplot
0,-1,architect lives wife unexpected death Eventual...
1,0,thunderstorm traveling circus accidentally lea...
2,1,killed revived blind martial arts teaches anci...
3,2,lives coach basketball team having played team...
4,3,Set backdrop film follows boy moves father job...
5,4,year old moved fictional small town father pre...
6,5,opening narration explains observed extraterre...
7,6,lieutenant assigned rescue man working secret ...


In [20]:
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer

def c_tf_idf(documents, m, ngram_range=(1, 1)):
    count = CountVectorizer(ngram_range=ngram_range, stop_words="english").fit(documents)
    t = count.transform(documents).toarray()
    w = t.sum(axis=1)
    tf = np.divide(t.T, w)
    sum_t = t.sum(axis=0)
    idf = np.log(np.divide(m, sum_t)).reshape(-1, 1)
    tf_idf = np.multiply(tf, idf)

    return tf_idf, count

def extract_top_n_words_per_topic(tf_idf, count, docs_per_topic, n=20):
    words = count.get_feature_names()
    labels = list(docs_per_topic.Topic)
    tf_idf_transposed = tf_idf.T
    indices = tf_idf_transposed.argsort()[:, -n:]
    top_n_words = {label: [(words[j], tf_idf_transposed[i][j]) for j in indices[i]][::-1] for i, label in enumerate(labels)}
    return top_n_words

def extract_topic_sizes(df):
    topic_sizes = (df.groupby(['Topic'])
                     .text
                     .count()
                     .reset_index()
                     .rename({"Topic": "Topic", "text": "Size"}, axis='columns')
                     .sort_values("Size", ascending=False))
    return topic_sizes

In [21]:
tf_idf_plots, count_plots = c_tf_idf(docs_df_movies.filteredplot.values, m=len(docs_df_movies))

In [22]:
top_n_words_plots = extract_top_n_words_per_topic(tf_idf_plots, count_plots, docs_df_movies, n=20)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [25]:
from pprint import pprint


def get_topic_index_description(top_n_words,ncount):
  topic_indexes = set(top_n_words.keys())
  topic_index_description = {}
  for topic_index in topic_indexes:
    desc = top_n_words[topic_index]
    desc = desc[:ncount]
    desc_string = [x[0] for x in desc]
    desc_string_mod = " | ".join(desc_string)
    topic_index_description[topic_index]  = desc_string_mod
  return topic_index_description

topic_index_description = get_topic_index_description(top_n_words_plots,8)



pprint (topic_index_description)

{-1: 'innovation | disrepair | corrections | validity | forsythia | valium | '
     'corresponds | gloomy',
 0: 'cuteness | crested | pairing | heck | marmosets | inator | superbark | '
    'scratte',
 1: 'eaters | tractionists | therru | ephialtes | herald | commoner | drago | '
    'elven',
 2: 'upfront | 214 | abbess | spirituality | heiter | insemination | fangirls '
    '| saverin',
 3: 'ulises | enfleurage | jaunt | tish | nanking | putt | swimmers | booger',
 4: 'masjid | raisa | aadi | ranveer | dissident | rial | dissidents | bol',
 5: 'elysium | fillings | crewmembers | tug | warhammer | orlean | '
    'overpopulation | zords',
 6: 'ss | mistrial | dunning | deado | controllers | wolfpack | obanno | 26s'}


In [26]:
topics_companies_desc = [topic_index_description[i] for i in df["cluster labels"]]
df["topic description"]  = topics_companies_desc

df.head()

,Unnamed: 0,Movie Name,Wiki Link,Plot,full_embeddings,umap_embeddings_5dim,umap_embeddings_2dim,cluster labels,filteredplot,topic description
0,0,White Noise,https://en.wikipedia.org//wiki/White_Noise_(20...,Jonathan Rivers is an architect and lives with...,"[-0.0726638212800026, 0.05636344477534294, 0.0...","[17.979957580566406, 0.06909535080194473, 14.6...","[17.152328491210938, -0.18462996184825897]",-1,architect lives wife unexpected death Eventual...,innovation | disrepair | corrections | validit...
1,1,Coach Carter,https://en.wikipedia.org//wiki/Coach_Carter,"Ken Carter lives in Richmond, California. He b...","[-0.0666189044713974, 0.04147614911198616, 0.0...","[19.398845672607422, 0.8634682297706604, 15.32...","[18.187257766723633, -0.751857340335846]",2,lives coach basketball team having played team...,upfront | 214 | abbess | spirituality | heiter...
2,2,Elektra,https://en.wikipedia.org//wiki/Elektra_(2005_f...,"After being killed in Daredevil, Elektra Natch...","[-0.03511177748441696, -0.01160984206944704, 0...","[18.37731170654297, -0.19560715556144714, 13.2...","[13.422314643859863, 0.4871552884578705]",1,killed revived blind martial arts teaches anci...,eaters | tractionists | therru | ephialtes | h...
3,3,Racing Stripes,https://en.wikipedia.org//wiki/Racing_Stripes,"During a thunderstorm, a traveling circus, Cir...","[-0.05486368015408516, 0.029759129509329796, -...","[19.10247039794922, 0.8358369469642639, 13.432...","[13.882101058959961, -0.6579449772834778]",0,thunderstorm traveling circus accidentally lea...,cuteness | crested | pairing | heck | marmoset...
4,4,Tom and Jerry: Blast Off to Mars,https://en.wikipedia.org//wiki/Tom_and_Jerry:_...,Tom (voiced by Bill Kopp) chases Jerry as usua...,"[0.010051962919533253, 0.06563024967908859, 0....","[18.755529403686523, 0.9130203127861023, 14.69...","[15.446535110473633, -2.076817035675049]",-1,voiced chases usual house town arrive astronau...,innovation | disrepair | corrections | validit...


In [27]:
import numpy as np
import plotly.express as px
# plotvalues = [[x1,x2,x3,..xn][y1,y2,y3,.....yn]] 
col="umap_embeddings_2dim"
color = "topic description"
hover_data =["Movie Name"]
hover_name = "Movie Name"
plot_values = np.stack(df[col], axis=1)


x, y = plot_values[0], plot_values[1]
fig = px.scatter(
            df,
            x=x,
            y=y,
            color=color,
            hover_data=hover_data,
            title="Movies",
            hover_name=hover_name,
        )

fig.show()

In [ ]:
options_to_choose = ["Action","Comedy","Asian","Romantic","Thriller","Drama","Fantasy","Mystery","Extraterrestrial","Adventure","Horror","Scifi","Superhero","Fiction","Animals","Other"]
print (options_to_choose)

In [30]:
import ipywidgets as widgets                        # Creating widgets
from IPython.display import display,HTML                 # Displaying widgets
# display(HTML("<style>.container { width:100% !important; }</style>"))
output = widgets.Output()
# Creating dropdown objects
dropdownobjects =[widgets.Dropdown(options = options_to_choose, description=str(key),width =75) for key in topic_index_description]

# Display the dropdowns
input_widgets = widgets.VBox(dropdownobjects)
pprint (topic_index_description)
print ("Choose from the dropdown the best replacement for a given word and proceed to the next cell : \n")
display(input_widgets)

{-1: 'innovation | disrepair | corrections | validity | forsythia | valium | '
     'corresponds | gloomy',
 0: 'cuteness | crested | pairing | heck | marmosets | inator | superbark | '
    'scratte',
 1: 'eaters | tractionists | therru | ephialtes | herald | commoner | drago | '
    'elven',
 2: 'upfront | 214 | abbess | spirituality | heiter | insemination | fangirls '
    '| saverin',
 3: 'ulises | enfleurage | jaunt | tish | nanking | putt | swimmers | booger',
 4: 'masjid | raisa | aadi | ranveer | dissident | rial | dissidents | bol',
 5: 'elysium | fillings | crewmembers | tug | warhammer | orlean | '
    'overpopulation | zords',
 6: 'ss | mistrial | dunning | deado | controllers | wolfpack | obanno | 26s'}
Choose from the dropdown the best replacement for a given word and proceed to the next cell : 



In [31]:
final_UI_chosen_mapping = {}
for key,choice in zip(topic_index_description.keys(),dropdownobjects):
  final_UI_chosen_mapping[key] = choice.value
pprint (final_UI_chosen_mapping)

{-1: 'Other',
 0: 'Animals',
 1: 'Adventure',
 2: 'Romantic',
 3: 'Superhero',
 4: 'Asian',
 5: 'Fiction',
 6: 'Horror'}


In [32]:
topics_grants_desc_short = [final_UI_chosen_mapping[i] for i in df["cluster labels"]]
df["topic final"]  = topics_grants_desc_short
df.head()

,Unnamed: 0,Movie Name,Wiki Link,Plot,full_embeddings,umap_embeddings_5dim,umap_embeddings_2dim,cluster labels,filteredplot,topic description,topic final
0,0,White Noise,https://en.wikipedia.org//wiki/White_Noise_(20...,Jonathan Rivers is an architect and lives with...,"[-0.0726638212800026, 0.05636344477534294, 0.0...","[17.979957580566406, 0.06909535080194473, 14.6...","[17.152328491210938, -0.18462996184825897]",-1,architect lives wife unexpected death Eventual...,innovation | disrepair | corrections | validit...,Other
1,1,Coach Carter,https://en.wikipedia.org//wiki/Coach_Carter,"Ken Carter lives in Richmond, California. He b...","[-0.0666189044713974, 0.04147614911198616, 0.0...","[19.398845672607422, 0.8634682297706604, 15.32...","[18.187257766723633, -0.751857340335846]",2,lives coach basketball team having played team...,upfront | 214 | abbess | spirituality | heiter...,Romantic
2,2,Elektra,https://en.wikipedia.org//wiki/Elektra_(2005_f...,"After being killed in Daredevil, Elektra Natch...","[-0.03511177748441696, -0.01160984206944704, 0...","[18.37731170654297, -0.19560715556144714, 13.2...","[13.422314643859863, 0.4871552884578705]",1,killed revived blind martial arts teaches anci...,eaters | tractionists | therru | ephialtes | h...,Adventure
3,3,Racing Stripes,https://en.wikipedia.org//wiki/Racing_Stripes,"During a thunderstorm, a traveling circus, Cir...","[-0.05486368015408516, 0.029759129509329796, -...","[19.10247039794922, 0.8358369469642639, 13.432...","[13.882101058959961, -0.6579449772834778]",0,thunderstorm traveling circus accidentally lea...,cuteness | crested | pairing | heck | marmoset...,Animals
4,4,Tom and Jerry: Blast Off to Mars,https://en.wikipedia.org//wiki/Tom_and_Jerry:_...,Tom (voiced by Bill Kopp) chases Jerry as usua...,"[0.010051962919533253, 0.06563024967908859, 0....","[18.755529403686523, 0.9130203127861023, 14.69...","[15.446535110473633, -2.076817035675049]",-1,voiced chases usual house town arrive astronau...,innovation | disrepair | corrections | validit...,Other


In [33]:
import numpy as np
import plotly.express as px
# plotvalues = [[x1,x2,x3,..xn][y1,y2,y3,.....yn]] 
col="umap_embeddings_2dim"
color = "topic final"
hover_data =["Movie Name"]
hover_name = "Movie Name"
plot_values = np.stack(df[col], axis=1)


x, y = plot_values[0], plot_values[1]
fig = px.scatter(
            df,
            x=x,
            y=y,
            color=color,
            hover_data=hover_data,
            title="Movies",
            hover_name=hover_name,
        )

fig.show()